## Importing Libraries and Initial Setup

The following libraries and modules are imported and initialized for text preprocessing and feature extraction:

`pandas`: Library for data manipulation and analysis.

`string`: Contains common string operations and constants.

`nltk`: The Natural Language Toolkit, a comprehensive library for natural language processing. Here, it is used for:

`nltk.tokenize.word_tokenize`: Tokenizes sentences into individual words (tokens).

`nltk.corpus.stopwords`: Provides a set of common stopwords in multiple languages. These stopwords are used for filtering irrelevant words (e.g., "the", "is") from the text.

`spacy`: Library for advanced natural language processing.

`sklearn.feature_extraction.text.TfidfVectorizer`: For converting text to TF-IDF features.

NLTK data (`punkt` and `stopwords`) is downloaded for tokenization and filtering out common stop words.

The `en_core_web_trf` model from spaCy is loaded for processing text with transformers specifically for lemmatization.

In [1]:
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')

nlp = spacy.load("en_core_web_trf")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Filipino Stopwords

The following list contains commonly used Filipino stopwords. These are words that are typically filtered out in text processing as they do not carry significant meaning in the context of text analysis.

These stopwords include common words such as articles, prepositions, and conjunctions that are usually removed during text preprocessing to focus on more meaningful words in text analysis.

In [2]:
filipino_stopwords = [
    "a", "ako", "ang", "ano", "at", "ay", "ibang", "ito", "iyon", "ka",
    "kami", "kanila", "kanya", "kayo", "laki", "mga", "na", "ng", "ni",
    "nito", "nang", "sa", "sila", "tayo", "walang", "yung", "si", "bawat",
    "kung", "hindi", "para", "dahil", "doon", "baka", "kapag", "saan",
    "sino", "siya", "tama", "yan", "o", "pala", "pero", "wala", "huwag",
    "muna", "na", "naman", "pag", "sana", "tulad", "upang", "bago", 
    "dati", "iba", "madami", "nakita", "pagkatapos", "pati", "sabi", "sana"
]

## Function: `load_dataset`

The `load_dataset` function is used to load a dataset from a CSV file. It handles various potential errors during the file loading process.

### Code

In [3]:
def load_dataset(file_path):
    try:
        data = pd.read_csv(file_path)
        print("Dataset loaded successfully!")
        return data
    except FileNotFoundError:
        print("File not found. Please check the file path.")
    except pd.errors.EmptyDataError:
        print("File is empty. Please check the file content.")
    except pd.errors.ParserError:
        print("Error parsing file. Please check the file format.")
    except Exception as e:
        print(f"An error occurred: {e}")

## Loading Dataset and Extracting Sentences

The following code snippet demonstrates how to load a dataset from a CSV file and extract sentences from it.

### Code

In [4]:
file_path = '../data/training_data_fil_sample.csv'
data = load_dataset(file_path)

sentences = data['sentence'].tolist()

Dataset loaded successfully!


## Function: `format_list_as_string`

The `format_list_as_string` function converts a list of tokens into a formatted string representation.

In text preprocessing, tokens are often stored as lists of words after steps like tokenization, stopword removal, and lemmatization. These lists can sometimes be difficult to read or display, especially when reviewing or debugging processed data.

this function ensures that these lists are formatted neatly when printed.

### Code

In [5]:
def format_list_as_string(token_list):
    return str(token_list).replace("'", '"')

## Function: `print_table`

The `print_table` function displays a formatted table using the `rich` library, which provides enhanced terminal output for data visualization.

### Code

In [6]:
def print_table(data, title="Table", num_samples=10):
    from rich.console import Console
    from rich.table import Table
    
    table = Table(title=title)
    
    for col in data.columns:
        table.add_column(col)

    for _, row in data.head(num_samples).iterrows():
        formatted_row = [format_list_as_string(row[col]) if isinstance(row[col], list) else row[col] for col in data.columns]
        table.add_row(*map(str, formatted_row))
    
    console = Console()
    console.print(table)

print_table(data, title="Original Data")

                                                   Original Data                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ "Okay lang ako! Ano ba kayo?"                                                                         │ anger   │
│ "Nasira ang iyong tibialis anterior."                                                                 │ fear    │
│ "Ha? Saan 'yun? Ay basta, ayos lang ako. Habulin na lang natin 'yung kriminal at baka makatakas       │ anger   │
│ siya!"                                                                                                │         │
│ "Nakakalungkot, nasa Kanluran siya."                                                                  │ sadness │
│ "Tsk. Kailangan kong magtago. Alam na ng mga pulis na ako ang pumatay sa hayup na 'yun," sabi nung    │ fear    │
│ kriminal.                                                                                             │         │
│ Ito na lang ang alaala ko sa kanya dahil . . . dahil . . . wala na rin si Demi na pusa niya.          │ sadness │
│ Lagi kong sinisisi ang sarili ko dahil sa pagkamatay ni Demi at dahil napakahina ko noon.             │ sadness │
│ Pero kapag naaalala ko 'yung sinabi nung Huntres habang nasa digmaan, lumalakas ang loob ko. Gusto    │ joy     │
│ kong maging katulad niya. Kahit nasa gitna kami ng laban nun ay wala siyang pinakitang kahinaan,      │         │
│ kahit na andaming namamatay sa paligid niya.                                                          │         │
│ "Sa burol na naman?" sabay ngiti sa akin ni Akane.                                                    │ joy     │
│ Napangiti tuloy ako bigla.                                                                            │ joy     │
└───────────────────────────────────────────────────────────────────────────────────────────────────────┴─────────┘

## Function: `convert_to_lowercase`

The `convert_to_lowercase` function converts all text in the 'sentence' column of a DataFrame to lowercase.

It ensures uniformity in text data by transforming all characters to lowercase, which is a key preprocessing step in natural language processing (NLP) tasks.

### Code

In [7]:
def convert_to_lowercase(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].str.lower()
        print("Sentence has been converted to lowercase.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

convert_to_lowercase(data)
print_table(data, title="Data After Lowercase Conversion")

Sentence has been converted to lowercase.


                                          Data After Lowercase Conversion                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ "okay lang ako! ano ba kayo?"                                                                         │ anger   │
│ "nasira ang iyong tibialis anterior."                                                                 │ fear    │
│ "ha? saan 'yun? ay basta, ayos lang ako. habulin na lang natin 'yung kriminal at baka makatakas       │ anger   │
│ siya!"                                                                                                │         │
│ "nakakalungkot, nasa kanluran siya."                                                                  │ sadness │
│ "tsk. kailangan kong magtago. alam na ng mga pulis na ako ang pumatay sa hayup na 'yun," sabi nung    │ fear    │
│ kriminal.                                                                                             │         │
│ ito na lang ang alaala ko sa kanya dahil . . . dahil . . . wala na rin si demi na pusa niya.          │ sadness │
│ lagi kong sinisisi ang sarili ko dahil sa pagkamatay ni demi at dahil napakahina ko noon.             │ sadness │
│ pero kapag naaalala ko 'yung sinabi nung huntres habang nasa digmaan, lumalakas ang loob ko. gusto    │ joy     │
│ kong maging katulad niya. kahit nasa gitna kami ng laban nun ay wala siyang pinakitang kahinaan,      │         │
│ kahit na andaming namamatay sa paligid niya.                                                          │         │
│ "sa burol na naman?" sabay ngiti sa akin ni akane.                                                    │ joy     │
│ napangiti tuloy ako bigla.                                                                            │ joy     │
└───────────────────────────────────────────────────────────────────────────────────────────────────────┴─────────┘

## Function: `remove_punctuation`

The `remove_punctuation` function removes punctuation from all text in the 'sentence' column of a DataFrame.

It ensures that only the meaningful words remain, free of any punctuation that could interfere with subsequent text processing steps.

### Code

In [8]:
def remove_punctuation(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
        print("Punctuation has been removed.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

remove_punctuation(data)
print_table(data, title="Data After Punctuation Removal")



Punctuation has been removed.


                                          Data After Punctuation Removal                                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ okay lang ako ano ba kayo                                                                             │ anger   │
│ nasira ang iyong tibialis anterior                                                                    │ fear    │
│ ha saan yun ay basta ayos lang ako habulin na lang natin yung kriminal at baka makatakas siya         │ anger   │
│ nakakalungkot nasa kanluran siya                                                                      │ sadness │
│ tsk kailangan kong magtago alam na ng mga pulis na ako ang pumatay sa hayup na yun sabi nung kriminal │ fear    │
│ ito na lang ang alaala ko sa kanya dahil    dahil    wala na rin si demi na pusa niya                 │ sadness │
│ lagi kong sinisisi ang sarili ko dahil sa pagkamatay ni demi at dahil napakahina ko noon              │ sadness │
│ pero kapag naaalala ko yung sinabi nung huntres habang nasa digmaan lumalakas ang loob ko gusto kong  │ joy     │
│ maging katulad niya kahit nasa gitna kami ng laban nun ay wala siyang pinakitang kahinaan kahit na    │         │
│ andaming namamatay sa paligid niya                                                                    │         │
│ sa burol na naman sabay ngiti sa akin ni akane                                                        │ joy     │
│ napangiti tuloy ako bigla                                                                             │ joy     │
└───────────────────────────────────────────────────────────────────────────────────────────────────────┴─────────┘

## Function: `remove_numbers`

The `remove_numbers` function removes numerical digits from all text in the 'sentence' column of a DataFrame.

It ensures that the text data is free from irrelevant numerical characters that could distort the analysis.

### Code

In [9]:
def remove_numbers(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].str.replace(r'\d+', '', regex=True)
        print("Numbers have been removed.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

remove_numbers(data)
print_table(data, title="Data After Numbers Removal")


Numbers have been removed.


                                            Data After Numbers Removal                                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ okay lang ako ano ba kayo                                                                             │ anger   │
│ nasira ang iyong tibialis anterior                                                                    │ fear    │
│ ha saan yun ay basta ayos lang ako habulin na lang natin yung kriminal at baka makatakas siya         │ anger   │
│ nakakalungkot nasa kanluran siya                                                                      │ sadness │
│ tsk kailangan kong magtago alam na ng mga pulis na ako ang pumatay sa hayup na yun sabi nung kriminal │ fear    │
│ ito na lang ang alaala ko sa kanya dahil    dahil    wala na rin si demi na pusa niya                 │ sadness │
│ lagi kong sinisisi ang sarili ko dahil sa pagkamatay ni demi at dahil napakahina ko noon              │ sadness │
│ pero kapag naaalala ko yung sinabi nung huntres habang nasa digmaan lumalakas ang loob ko gusto kong  │ joy     │
│ maging katulad niya kahit nasa gitna kami ng laban nun ay wala siyang pinakitang kahinaan kahit na    │         │
│ andaming namamatay sa paligid niya                                                                    │         │
│ sa burol na naman sabay ngiti sa akin ni akane                                                        │ joy     │
│ napangiti tuloy ako bigla                                                                             │ joy     │
└───────────────────────────────────────────────────────────────────────────────────────────────────────┴─────────┘

## Function: `tokenize_sentences`

The `tokenize_sentences` function tokenizes each sentence in the 'sentence' column of a DataFrame into individual words.

It breaks down sentences into individual words or tokens, facilitating analysis, standardizing input, and enabling various natural language processing tasks.

### Code

In [10]:
def tokenize_sentences(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda x: word_tokenize(x))
        print("Sentences have been tokenized.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

tokenize_sentences(data)
print_table(data, title="Data After Tokenization")

Sentences have been tokenized.


                                              Data After Tokenization                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ ["okay", "lang", "ako", "ano", "ba", "kayo"]                                                          │ anger   │
│ ["nasira", "ang", "iyong", "tibialis", "anterior"]                                                    │ fear    │
│ ["ha", "saan", "yun", "ay", "basta", "ayos", "lang", "ako", "habulin", "na", "lang", "natin", "yung", │ anger   │
│ "kriminal", "at", "baka", "makatakas", "siya"]                                                        │         │
│ ["nakakalungkot", "nasa", "kanluran", "siya"]                                                         │ sadness │
│ ["tsk", "kailangan", "kong", "magtago", "alam", "na", "ng", "mga", "pulis", "na", "ako", "ang",       │ fear    │
│ "pumatay", "sa", "hayup", "na", "yun", "sabi", "nung", "kriminal"]                                    │         │
│ ["ito", "na", "lang", "ang", "alaala", "ko", "sa", "kanya", "dahil", "dahil", "wala", "na", "rin",    │ sadness │
│ "si", "demi", "na", "pusa", "niya"]                                                                   │         │
│ ["lagi", "kong", "sinisisi", "ang", "sarili", "ko", "dahil", "sa", "pagkamatay", "ni", "demi", "at",  │ sadness │
│ "dahil", "napakahina", "ko", "noon"]                                                                  │         │
│ ["pero", "kapag", "naaalala", "ko", "yung", "sinabi", "nung", "huntres", "habang", "nasa", "digmaan", │ joy     │
│ "lumalakas", "ang", "loob", "ko", "gusto", "kong", "maging", "katulad", "niya", "kahit", "nasa",      │         │
│ "gitna", "kami", "ng", "laban", "nun", "ay", "wala", "siyang", "pinakitang", "kahinaan", "kahit",     │         │
│ "na", "andaming", "namamatay", "sa", "paligid", "niya"]                                               │         │
│ ["sa", "burol", "na", "naman", "sabay", "ngiti", "sa", "akin", "ni", "akane"]                         │ joy     │
│ ["napangiti", "tuloy", "ako", "bigla"]                                                                │ joy     │
└───────────────────────────────────────────────────────────────────────────────────────────────────────┴─────────┘

## Function: `remove_stopwords`

The `remove_stopwords` function removes stopwords from each tokenized sentence in the 'sentence' column of a DataFrame.

Stopwords are common words that usually add little meaning to a sentence and are often filtered out in text processing to enhance the focus on more meaningful words.

### Code

In [11]:
def remove_stopwords(data):
    if 'sentence' in data.columns:
        english_stopwords = set(stopwords.words('english'))
        all_stopwords = english_stopwords.union(set(filipino_stopwords))
        
        data['sentence'] = data['sentence'].apply(lambda tokens: [word for word in tokens if word.lower() not in all_stopwords])
        print("Stopwords have been removed.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

remove_stopwords(data)
print_table(data, title="Data After Stopwords Removal")

Stopwords have been removed.


                                           Data After Stopwords Removal                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ ["okay", "lang", "ba"]                                                                                │ anger   │
│ ["nasira", "iyong", "tibialis", "anterior"]                                                           │ fear    │
│ ["ha", "yun", "basta", "ayos", "lang", "habulin", "lang", "natin", "kriminal", "makatakas"]           │ anger   │
│ ["nakakalungkot", "nasa", "kanluran"]                                                                 │ sadness │
│ ["tsk", "kailangan", "kong", "magtago", "alam", "pulis", "pumatay", "hayup", "yun", "nung",           │ fear    │
│ "kriminal"]                                                                                           │         │
│ ["lang", "alaala", "ko", "rin", "demi", "pusa", "niya"]                                               │ sadness │
│ ["lagi", "kong", "sinisisi", "sarili", "ko", "pagkamatay", "demi", "napakahina", "ko", "noon"]        │ sadness │
│ ["naaalala", "ko", "sinabi", "nung", "huntres", "habang", "nasa", "digmaan", "lumalakas", "loob",     │ joy     │
│ "ko", "gusto", "kong", "maging", "katulad", "niya", "kahit", "nasa", "gitna", "laban", "nun",         │         │
│ "siyang", "pinakitang", "kahinaan", "kahit", "andaming", "namamatay", "paligid", "niya"]              │         │
│ ["burol", "sabay", "ngiti", "akin", "akane"]                                                          │ joy     │
│ ["napangiti", "tuloy", "bigla"]                                                                       │ joy     │
└───────────────────────────────────────────────────────────────────────────────────────────────────────┴─────────┘

## Function: `lemmatize_tokens`

The `lemmatize_tokens` function lemmatizes each token in the 'sentence' column of a DataFrame using spaCy's lemmatization.

it reduces words to their base or dictionary form (lemmas), which helps in standardizing the text by consolidating different inflections of a word into a single representation, thereby improving the performance of natural language processing tasks such as classification and clustering.

### Code

In [12]:
def lemmatize_tokens(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda tokens: [nlp(token)[0].lemma_ for token in tokens])
        print("Tokens have been lemmatized.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

lemmatize_tokens(data)
print_table(data, title="Data After Lemmatization")

Tokens have been lemmatized.


                                             Data After Lemmatization                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ ["okay", "lang", "ba"]                                                                                │ anger   │
│ ["nasira", "iyong", "tibialis", "anterior"]                                                           │ fear    │
│ ["ha", "yun", "basta", "ayos", "lang", "habulin", "lang", "natin", "kriminal", "makatakas"]           │ anger   │
│ ["nakakalungkot", "nasa", "kanluran"]                                                                 │ sadness │
│ ["tsk", "kailangan", "kong", "magtago", "alam", "pulis", "pumatay", "hayup", "yun", "nung",           │ fear    │
│ "kriminal"]                                                                                           │         │
│ ["lang", "alaala", "ko", "rin", "demi", "pusa", "niya"]                                               │ sadness │
│ ["lagi", "kong", "sinisisi", "sarili", "ko", "pagkamatay", "demi", "napakahina", "ko", "noon"]        │ sadness │
│ ["naaalala", "ko", "sinabi", "nung", "huntres", "habang", "nasa", "digmaan", "lumalakas", "loob",     │ joy     │
│ "ko", "gusto", "kong", "mage", "katulad", "niya", "kahit", "nasa", "gitna", "laban", "nun", "siyang", │         │
│ "pinakitang", "kahinaan", "kahit", "andame", "namamatay", "paligid", "niya"]                          │         │
│ ["burol", "sabay", "ngiti", "akin", "akane"]                                                          │ joy     │
│ ["napangiti", "tuloy", "bigla"]                                                                       │ joy     │
└───────────────────────────────────────────────────────────────────────────────────────────────────────┴─────────┘

## Function: `join_tokens`

The `join_tokens` function joins tokens in each list within the 'sentence' column of a DataFrame back into single sentences.

it reconstructs the original text format after various transformations, enabling further analysis or modeling tasks that require complete sentences, such as text classification, sentiment analysis, or readability assessments.

### Code

In [13]:
def join_tokens(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda tokens: ' '.join(tokens))
        print("Tokens have been joined back into sentences.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

join_tokens(data)
print_table(data, title="Data After Joining Tokens")

Tokens have been joined back into sentences.


                                             Data After Joining Tokens                                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ okay lang ba                                                                                          │ anger   │
│ nasira iyong tibialis anterior                                                                        │ fear    │
│ ha yun basta ayos lang habulin lang natin kriminal makatakas                                          │ anger   │
│ nakakalungkot nasa kanluran                                                                           │ sadness │
│ tsk kailangan kong magtago alam pulis pumatay hayup yun nung kriminal                                 │ fear    │
│ lang alaala ko rin demi pusa niya                                                                     │ sadness │
│ lagi kong sinisisi sarili ko pagkamatay demi napakahina ko noon                                       │ sadness │
│ naaalala ko sinabi nung huntres habang nasa digmaan lumalakas loob ko gusto kong mage katulad niya    │ joy     │
│ kahit nasa gitna laban nun siyang pinakitang kahinaan kahit andame namamatay paligid niya             │         │
│ burol sabay ngiti akin akane                                                                          │ joy     │
│ napangiti tuloy bigla                                                                                 │ joy     │
└───────────────────────────────────────────────────────────────────────────────────────────────────────┴─────────┘

## Function: `vectorize_with_tfidf`

The `vectorize_with_tfidf` function performs TF-IDF vectorization on the 'sentence' column of a DataFrame.

**TF-IDF** is a statistical measure used to evaluate the importance of a word in a document relative to a collection (or corpus) of documents. It combines two key concepts:

1. **Term Frequency (TF)**: Measures how frequently a word appears in a single document. Words that appear frequently within a document are considered more important for that document.
   - Formula: `TF = (Number of times a term appears in a document) / (Total number of terms in the document)`

2. **Inverse Document Frequency (IDF)**: Measures how important a word is across the entire dataset. Words that appear in many documents (common words) are less important, while rare words are more important.
   - Formula: `IDF = log(Total number of documents / Number of documents containing the term)`

The **TF-IDF score** is calculated by multiplying TF and IDF for each word, providing a numerical value representing the word's significance within a document relative to the entire corpus.

### Code

In [14]:
def vectorize_with_tfidf(data):
    if 'sentence' in data.columns:
        vectorizer = TfidfVectorizer()

        tfidf_matrix = vectorizer.fit_transform(data['sentence'])

        print("TF-IDF Vectorization complete.")
        return tfidf_matrix, vectorizer
    else:
        print("Column 'sentence' not found in the DataFrame.")

tfidf_matrix, vectorizer = vectorize_with_tfidf(data)
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
tfidf_df['emotion'] = data['emotion'].values

print(tfidf_df.head())

TF-IDF Vectorization complete.
   akane  akin  alaala      alam  andame  anterior      ayos       ba  \
0    0.0   0.0     0.0  0.000000     0.0       0.0  0.000000  0.62584   
1    0.0   0.0     0.0  0.000000     0.0       0.5  0.000000  0.00000   
2    0.0   0.0     0.0  0.000000     0.0       0.0  0.321781  0.00000   
3    0.0   0.0     0.0  0.000000     0.0       0.0  0.000000  0.00000   
4    0.0   0.0     0.0  0.320732     0.0       0.0  0.000000  0.00000   

      basta  bigla  ...  sabay  sarili  sinabi  sinisisi  siyang  tibialis  \
0  0.000000    0.0  ...    0.0     0.0     0.0       0.0     0.0       0.0   
1  0.000000    0.0  ...    0.0     0.0     0.0       0.0     0.0       0.5   
2  0.321781    0.0  ...    0.0     0.0     0.0       0.0     0.0       0.0   
3  0.000000    0.0  ...    0.0     0.0     0.0       0.0     0.0       0.0   
4  0.000000    0.0  ...    0.0     0.0     0.0       0.0     0.0       0.0   

        tsk  tuloy       yun  emotion  
0  0.000000    0.0  0